In [16]:
import random
import collections

    
def SRAP():
    """This agent takes action based solely on the percept"""
    def program(percept):
        loc, status = percept
        return ('Suck' if status == 'Dirty'
                else 'Right' if loc == loc_A else 'Left')
    return program
def homework(default_env,default_agent,is_print=False):
    #Only two squares in the vaccum environment
    loc_A = (0, 0)
    loc_B = (1, 0)
    loc_dict = loc_A, loc_B
    
    # Agent
    simple_agent = Agent(SRAP())
    
    # Initialize the two-state environment
    trivial_vacuum_env = TrivialVacuumEnvironment()
    trivial_vacuum_env.status = default_env
    print("Default State of the Environment: {}.".format(trivial_vacuum_env.status))
    trivial_vacuum_env.add_thing(simple_agent)
    simple_agent.location = default_agent
    print("Default SimpleReflexVacuumAgent is located at {}.".format(simple_agent.location))
    awards = 0

    for i in range(1000):
        for loc in loc_dict:
            if trivial_vacuum_env.status[loc] == 'Clean': awards += 1
        # Run
        trivial_vacuum_env.step()
        # Print the current state of the environment
        if is_print == True:
            print("Time step: {}".format(i+1))
            print("State of the Environment: {}.".format(trivial_vacuum_env.status))
            print("SimpleReflexVacuumAgent is located at {}.".format(simple_agent.location))
    print("performance score is " + str(awards))

In [17]:
class Thing:
    """This represents any physical object that can appear in an Environment.
    You subclass Thing to get the things you want. Each thing can have a
    .__name__  slot (used for output only)."""

    def __repr__(self):
        return '<{}>'.format(getattr(self, '__name__', self.__class__.__name__))

    def is_alive(self):
        """Things that are 'alive' should return true."""
        return hasattr(self, 'alive') and self.alive

    def show_state(self):
        """Display the agent's internal state. Subclasses should override."""
        print("I don't know how to show_state.")

    def display(self, canvas, x, y, width, height):
        """Display an image of this Thing on the canvas."""
        # Do we need this?
        pass


class Agent(Thing):
    """An Agent is a subclass of Thing with one required slot,
    .program, which should hold a function that takes one argument, the
    percept, and returns an action. (What counts as a percept or action
    will depend on the specific environment in which the agent exists.)
    Note that 'program' is a slot, not a method. If it were a method,
    then the program could 'cheat' and look at aspects of the agent.
    It's not supposed to do that: the program can only look at the
    percepts. An agent program that needs a model of the world (and of
    the agent itself) will have to build and maintain its own model.
    There is an optional slot, .performance, which is a number giving
    the performance measure of the agent in its environment."""

    def __init__(self, program=None):
        self.alive = True
        self.bump = False
        self.holding = []
        self.performance = 0
        if program is None or not isinstance(program, collections.Callable):
            print("Can't find a valid program for {}, falling back to default.".format(
                self.__class__.__name__))

            def program(percept):
                return eval(input('Percept={}; action? '.format(percept)))

        self.program = program

    def can_grab(self, thing):
        """Return True if this agent can grab this thing.
        Override for appropriate subclasses of Agent and Thing."""
        return False

# ______________________________________________________________________________


loc_A, loc_B = (0, 0), (1, 0)  # The two locations for the Vacuum world

class Environment:
    """Abstract class representing an Environment. 'Real' Environment classes
    inherit from this. Your Environment will typically need to implement:
        percept:           Define the percept that an agent sees.
        execute_action:    Define the effects of executing an action.
                           Also update the agent.performance slot.
    The environment keeps a list of .things and .agents (which is a subset
    of .things). Each agent has a .performance slot, initialized to 0.
    Each thing has a .location slot, even though some environments may not
    need this."""

    def __init__(self):
        self.things = []
        self.agents = []

    def thing_classes(self):
        return []  # List of classes that can go into environment

    def percept(self, agent):
        """Return the percept that the agent sees at this point. (Implement this.)"""
        raise NotImplementedError

    def execute_action(self, agent, action):
        """Change the world to reflect this action. (Implement this.)"""
        raise NotImplementedError

    def default_location(self, thing):
        """Default location to place a new thing with unspecified location."""
        return None

    def exogenous_change(self):
        """If there is spontaneous change in the world, override this."""
        pass

    def is_done(self):
        """By default, we're done when we can't find a live agent."""
        return not any(agent.is_alive() for agent in self.agents)

    def step(self):
        """Run the environment for one time step. If the
        actions and exogenous changes are independent, this method will
        do. If there are interactions between them, you'll need to
        override this method."""
        if not self.is_done():
            actions = []
            for agent in self.agents:
                if agent.alive:
                    actions.append(agent.program(self.percept(agent)))
                else:
                    actions.append("")
            for (agent, action) in zip(self.agents, actions):
                self.execute_action(agent, action)
            self.exogenous_change()

    def run(self, steps=1000):
        """Run the Environment for given number of time steps."""
        for step in range(steps):
            if self.is_done():
                return
            self.step()


    def add_thing(self, thing, location=None):
        """Add a thing to the environment, setting its location. For
        convenience, if thing is an agent program we make a new agent
        for it. (Shouldn't need to override this.)"""
        if not isinstance(thing, Thing):
            thing = Agent(thing)
        if thing in self.things:
            print("Can't add the same thing twice")
        else:
            thing.location = location if location is not None else self.default_location(thing)
            self.things.append(thing)
            if isinstance(thing, Agent):
                thing.performance = 0
                self.agents.append(thing)

    def delete_thing(self, thing):
        """Remove a thing from the environment."""
        try:
            self.things.remove(thing)
        except ValueError as e:
            print(e)
            print("  in Environment delete_thing")
            print("  Thing to be removed: {} at {}".format(thing, thing.location))
            print("  from list: {}".format([(thing, thing.location) for thing in self.things]))
        if thing in self.agents:
            self.agents.remove(thing)
            
class TrivialVacuumEnvironment(Environment):
    """This environment has two locations, A and B. Each can be Dirty
    or Clean. The agent perceives its location and the location's
    status. This serves as an example of how to implement a simple
    Environment."""

    def __init__(self):
        super().__init__()
        self.status = {loc_A: 'Dirty', loc_B: 'Dirty'}
        self.default_agent = loc_B
    def percept(self, agent):
        """Returns the agent's location, and the location status (Dirty/Clean)."""
        return (agent.location, self.status[agent.location])

    def execute_action(self, agent, action):
        """Change agent's location and/or location's status; track performance.
        Score 10 for each dirt cleaned; -1 for each move."""
        if action == 'Right':
            agent.location = loc_B
        elif action == 'Left':
            agent.location = loc_A
        elif action == 'Suck':
            if self.status[agent.location] == 'Dirty':
                self.status[agent.location] = 'Clean'

    def default_location(self, thing):
        return self.default_agent
            



In [18]:
loc_A = (0, 0)
loc_B = (1, 0)
env_df = {loc_A: 'Dirty', loc_B: 'Dirty'}
agent_df = loc_A
homework(env_df,agent_df)

Default State of the Environment: {(0, 0): 'Dirty', (1, 0): 'Dirty'}.
Default SimpleReflexVacuumAgent is located at (0, 0).
performance score is 1996


In [19]:
env_df = {loc_A: 'Dirty', loc_B: 'Dirty'}
agent_df = loc_B
homework(env_df,agent_df)

Default State of the Environment: {(0, 0): 'Dirty', (1, 0): 'Dirty'}.
Default SimpleReflexVacuumAgent is located at (1, 0).
performance score is 1996


In [20]:
env_df = {loc_A: 'Dirty', loc_B: 'Clean'}
agent_df = loc_A
homework(env_df,agent_df)

Default State of the Environment: {(0, 0): 'Dirty', (1, 0): 'Clean'}.
Default SimpleReflexVacuumAgent is located at (0, 0).
performance score is 1999


In [21]:
env_df = {loc_A: 'Dirty', loc_B: 'Clean'}
agent_df = loc_B
homework(env_df,agent_df)

Default State of the Environment: {(0, 0): 'Dirty', (1, 0): 'Clean'}.
Default SimpleReflexVacuumAgent is located at (1, 0).
performance score is 1998


In [22]:
env_df = {loc_A: 'Clean', loc_B: 'Dirty'}
agent_df = loc_A
homework(env_df,agent_df)

Default State of the Environment: {(0, 0): 'Clean', (1, 0): 'Dirty'}.
Default SimpleReflexVacuumAgent is located at (0, 0).
performance score is 1998


In [23]:
env_df = {loc_A: 'Clean', loc_B: 'Dirty'}
agent_df = loc_B
homework(env_df,agent_df)

Default State of the Environment: {(0, 0): 'Clean', (1, 0): 'Dirty'}.
Default SimpleReflexVacuumAgent is located at (1, 0).
performance score is 1999


In [24]:
env_df = {loc_A: 'Clean', loc_B: 'Clean'}
agent_df = loc_A
homework(env_df,agent_df)

Default State of the Environment: {(0, 0): 'Clean', (1, 0): 'Clean'}.
Default SimpleReflexVacuumAgent is located at (0, 0).
performance score is 2000


In [25]:
env_df = {loc_A: 'Clean', loc_B: 'Clean'}
agent_df = loc_B
homework(env_df,agent_df)

Default State of the Environment: {(0, 0): 'Clean', (1, 0): 'Clean'}.
Default SimpleReflexVacuumAgent is located at (1, 0).
performance score is 2000
